In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from behavior.selfmotionddm import SelfMotionDDM
from behavior.Accumulator import Accumulator

In [ ]:
# Initialize grid vectors for diffusion particle

grid_vec = np.arange(-4, 0, 0.05)  # -3 to 0, steps of 0.01
time_vec = np.arange(0, 2, 0.05)   # 0 to 2s, steps of 50ms

In [ ]:
drifts = [0, 0.1, 0.25, 0.5, 1., 2.]
accum = Accumulator(
    grid_vec=grid_vec,
    tvec=time_vec,
    bound=1,
)
accum.apply_drifts(drifts, labels=drifts)
accum.compute_distrs(return_pdf=True)
accum.plot()

In [ ]:
# NOTE on kmult:
# three-element list --> independent sensitivites for ves, low coh vis and high coh vis
# two-element list --> independent sensitivites for ves and vis. low coh and high coh proportional based on values
# one-element --> low coh and high coh proportional based on values, mean(low, high) = ves
#
# all other init_params values are lists, but can be:
# single element - same value for all modalities
# three elements - independent value per modality    

init_params = {
    'kmult': [1, 1],            # ves, vis sensitivites
    'bound': [1, 1, 1],         # ves, vis, comb bounds
    'non_dec_time': [0.3],      # non-decision time (secs)
    'wager_thr': [1, 1, 1],     # log odds threshold for high bets
    'wager_alpha': [0.05],      # base rate of high bets
}

# initialize DDM object for fitting
ddm = SelfMotionDDM(
    grid_vec=grid_vec,
    tvec=time_vec,
    **init_params, 
    stim_scaling=True,      # scale ves/vis according to acc/vel signals
    return_wager=True       # whether to compute pdfs and log odds maps, and return wagers
    )


